In [15]:
import numpy as np
import pandas as pd
np.random.seed(2)

from sklearn import datasets
iris = datasets.load_iris()
list(iris.keys())

['data', 'target', 'target_names', 'DESCR', 'feature_names']

In [10]:
print(iris['DESCR'])

Iris Plants Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20  0.76     0.9565  (high!)
    ============== ==== ==== ======= ===== ====================

    :Missing Attribute Values: None
  

In [9]:
X = iris['data']
y = iris['target']

# Add bias term
X = np.c_[np.ones([len(X),1]),X]

In [17]:
# split training and testing set
test_ratio = 0.2
validation_ratio = 0.2


total_size = len(X)
test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
training_size = total_size - validation_size - test_size

rnd_indices = np.random.permutation(total_size)

X_train = X[rnd_indices[:training_size]]
X_valid = X[rnd_indices[training_size:-test_size]]
X_test = X[rnd_indices[-test_size:]]

y_train = y[rnd_indices[:training_size]]
y_valid = y[rnd_indices[training_size:-test_size]]
y_test = y[rnd_indices[-test_size:]]

Convert the target indices to one-hot vector.

In [19]:
def to_one_hot(y):
    l = len(y)
    w = y.max() + 1
    y_one_hot = np.zeros([l,w])
    y_one_hot[np.arange(l),y] = 1
    return y_one_hot

In [22]:
y_train_one_hot = to_one_hot(y_train)
y_test_one_hot = to_one_hot(y_test)
y_valid_one_hot = to_one_hot(y_valid)

In [23]:
n_inputs = X_train.shape[1] 
n_outputs = len(np.unique(y_train))

Now let's implement the Softmax function. Recall that it is defined by the following equation:

$\sigma\left(\mathbf{s}(\mathbf{x})\right)_k = \dfrac{\exp\left(s_k(\mathbf{x})\right)}{\sum\limits_{j=1}^{K}{\exp\left(s_j(\mathbf{x})\right)}}$

In [25]:
def softmax(logit):
    exp = np.exp(logit)
    exp_sum = np.sum(exp, axis = 1, keepdims=True)
    return exp/exp_sum

So the equations we will need are the cost function:

$J(\mathbf{\Theta}) =
- \dfrac{1}{m}\sum\limits_{i=1}^{m}\sum\limits_{k=1}^{K}{y_k^{(i)}\log\left(\hat{p}_k^{(i)}\right)}$

And the equation for the gradients:

$\nabla_{\mathbf{\theta}^{(k)}} \, J(\mathbf{\Theta}) = \dfrac{1}{m} \sum\limits_{i=1}^{m}{ \left ( \hat{p}^{(i)}_k - y_k^{(i)} \right ) \mathbf{x}^{(i)}}$

Note that $\log\left(\hat{p}_k^{(i)}\right)$ may not be computable if $\hat{p}_k^{(i)} = 0$. So we will add a tiny value $\epsilon$ to $\log\left(\hat{p}_k^{(i)}\right)$ to avoid getting `nan` values.

In [38]:
eta = 0.01
n_iteration = 5001
m = len(X_train)
epsilon = 1e-7

Theta = np.random.randn(n_inputs,n_outputs)

for iteration in range(n_iteration):
    logit = X_train.dot(Theta)
    Y_proba = softmax(logit)
    loss = -np.mean(np.sum(y_train_one_hot * np.log(Y_proba + epsilon),axis = 1))
    error = Y_proba - y_train_one_hot
    if iteration % 500 == 0:
        print(iteration, loss)
    gradients = (1/m)*X_train.T.dot(error)
    Theta = Theta - eta * gradients

0 8.653886543767213
500 0.4227199470067875
1000 0.33540445846405337
1500 0.28652833712282216
2000 0.2522252200543796
2500 0.22634128826318686
3000 0.20601864023603383
3500 0.18960864146425824
4000 0.17606422538268146
4500 0.16468269896818305
5000 0.1549739327348464


In [39]:
Theta

array([[ 1.94759504,  1.77935789,  0.02876956],
       [ 1.31340542,  1.32633464, -0.83370289],
       [ 1.286319  , -0.65622922, -1.17575932],
       [-2.62493793, -0.77153175,  2.11152552],
       [-1.90467011, -0.84793181,  0.80632914]])

In [40]:
# make perdiction for the validation set and check the accuracy score
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9333333333333333

In [43]:
# add l2 norm
eta = 0.1
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1  # regularization hyperparameter

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    xentropy_loss = -np.mean(np.sum(y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
    loss = xentropy_loss + alpha * l2_loss
    error = Y_proba - y_train_one_hot
    if iteration % 500 == 0:
        print(iteration, loss)
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients

0 6.780055192727403
500 0.5785830917297082
1000 0.5410221538207446
1500 0.5114010115970132
2000 0.4888224384904699
2500 0.4768613532146928
3000 0.47273355601592026
3500 0.4697087350786935
4000 0.46746947107666864
4500 0.46580328700590795
5000 0.46455844725526574


In [44]:
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9333333333333333